<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/04_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 単語ベクトル


## 今日のお題
* 単語ベクトルを利用して、テキストをベクトル化する。
* ベクトル化を埋め込み(embedding)と呼ぶ。
  * 以下、埋め込みという言い方を使う。
* こうして作ったベクトルを使って、テキスト分類問題を解く。
* 同じ分類問題を、BERTでテキストをembedすることによって解く。
* 両者の性能を比較する。

## 単語ベクトルとは
* いわゆるword2vec。
 * https://arxiv.org/abs/1301.3781
 * https://en.wikipedia.org/wiki/Word2vec
* 単語をベクトルとして表現したもの。
 * 単語埋め込み、単語分散表現、などとも言われる。
* 意味が近い単語はベクトルとしても近くなるように、作成されている。


## 単語ベクトルを作るアルゴリズム
* アルゴリズム自体の説明は、この授業では割愛します。
 * https://www.tensorflow.org/text/tutorials/word2vec
* 大雑把には・・・
 * テキストをたくさん集める。それらのテキストの中で・・・
 * 各単語について、前後にどのような単語が出現するか、調べる。
 * 前後に似たような単語が出現する単語は、似たようなベクトルにマッピングする。

## 単語ベクトルの使いみち

### 単語の類似度評価
* ベクトルどうしの遠い近いを表す尺度は何でも使える。
* 内積やコサイン類似度が使われることが多い。


### テキスト埋め込み
* 最もシンプルには、テキストに含まれるトークンの単語ベクトルの平均を取ればよい。
  * これをmean poolingと呼ぶ。
* 単語ベクトルを使ってテキストを埋め込むことは、最近は行わない。
  * テキストのembedには、今は、深層学習言語モデルを使う。
  * 今回はsentence BERTを使った方法を紹介する。

## 準備

### 環境設定
* 今回はランタイムのタイプでGPUを選んでおいてください。
  * あとでBERTによるテキスト埋め込みと比較するため。

### 必要なライブラリのインストール

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.6 MB/s eta 0:00:00


## データセット

### WRIME: 主観と客観の感情分析データセット
* 詳細は、以下を参照。
  * https://github.com/ids-cv/wrime
* 短いテキストがたくさん含まれている。
* -2, -1, 0, 1, 2の５段階でnegativeからpositiveの感情ラベルが付与されている。
* 今回は、Hugging Face Hubからこのデータセットを取得する。


In [2]:
from datasets import load_dataset

dataset = load_dataset("shunk031/wrime", "ver2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

/root/.cache/huggingface/modules/datasets_modules/datasets/shunk031--wrime/4b2571d8c51503a1134ff7edb2293a2ccb16632689d2f064b8afda72ec9e01ce/wrime.py:80: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(". ", "_")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

* はじめから、train, validation, testの3つの集合に分けられている。

In [3]:
import numpy as np

tags = ["train", "validation", "test"]

texts = {}
labels = {}
for tag in tags:
  texts[tag] = dataset[tag]["sentence"]
  labels[tag] = [item["sentiment"] for item in dataset[tag]["avg_readers"]]
  labels[tag] = np.array(labels[tag])

In [4]:
texts["train"][0]

'ぼけっとしてたらこんな時間。チャリあるから食べにでたいのに…'

In [5]:
labels["train"][0]

-1

## 単語ベクトルによるテキストの埋め込み
* 小規模のモデル（名前が__`_sm`__で終わるモデル）は単語ベクトルを含まない。
* 大規模モデルはダウンロードに時間がかかる。
* そのため、中規模モデルをインストールする。

### 日本語中規模モデルのインストール
* https://spacy.io/models/ja#ja_core_news_md

In [6]:
!python -m spacy download ja_core_news_md

2023-10-13 11:38:34.530691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 11:38:36.833217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-13 11:38:38.383284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-13 11:38:38.383891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

### テキストの埋め込み
* spaCyではテキストを直接embedできる。
 * 内部では単語ベクトルの平均を求めている。
* （おそらく6分ぐらいかかります。）

In [7]:
from tqdm import tqdm
import numpy as np
import spacy

nlp = spacy.load('ja_core_news_md')

X = {}
for tag in tags:
  X[tag] = []
  for text in tqdm(texts[tag]):
    tokens = nlp(text)
    X[tag].append(tokens.vector)
  X[tag] = np.array(X[tag])

100%|██████████| 2500/2500 [00:26<00:00, 95.03it/s]


In [8]:
X["train"].shape

(30000, 300)

* embedした結果とラベルを保存しておく。

In [9]:
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'wb') as f:
    np.save(f, X[tag])
  with open(f'wrime_{tag}_label.npy', 'wb') as f:
    np.save(f, labels[tag])

### ラベルの前処理

* 保存しておいたテキストのベクトル表現とラベルを読み込む。

In [10]:
import numpy as np

tags = ["train", "validation", "test"]

X = {}
labels = {}
for tag in tags:
  with open(f'wrime_{tag}_vec.npy', 'rb') as f:
    X[tag] = np.load(f)
  with open(f'wrime_{tag}_label.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [11]:
X["train"].shape

(30000, 300)

In [12]:
labels["train"].shape

(30000,)

* 今回は、データセットのラベルを2値に単純化する
  * ラベル0のテキストは取り除く。
  * negativeを示す-2と-1は、一つのクラスにまとめる。
  * positiveを示す1と2も、一つのクラスにまとめる。

In [24]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

### 文書分類

* 適宜、チューニングしてください。
* 分類手法は`LinearSVC`でなくても構いません。

In [14]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

0.7686567164179104

## BERTによるテキストの埋め込み
* BERTにはいろいろな種類がある。
* 今日は、sentence BERTと呼ばれるBERTを使う。
* sentence BERTの説明は、今日はしない。とりあえず使う。
  * 単にテキストをembedするツールとして使う。

### 必要なライブラリのインストール

In [15]:
!pip install -q transformers fugashi[unidic-lite]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.4 MB/s eta 0:00:00


In [16]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=cac28e4f3b37cd5b737e37f8b6aef8327c25032e98547def2205e7924f225b43
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


### sentence BERTのロード
* 初回だけダウンロードに時間がかかる。
* 2回目以降は、ローカルに保存したモデルをロードするだけ。

In [17]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("cl-tohoku/bert-base-japanese-v3")

(…)9c77309f217bd7b1a79d43c7e/.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

(…)29b969c77309f217bd7b1a79d43c7e/README.md:   0%|          | 0.00/2.66k [00:00<?, ?B/s]

(…)b969c77309f217bd7b1a79d43c7e/config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

(…)f217bd7b1a79d43c7e/tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

(…)29b969c77309f217bd7b1a79d43c7e/vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

### テキストの埋め込み
* 内部では、BERTの出力のmean pooling
  * 詳細は、今日のところは、割愛します。

* （おそらく2分ぐらいで終わります。）

In [18]:
X = {}
for tag in tags:
  X[tag] = embedder.encode(texts[tag])

* embedした結果を保存しておく。

In [19]:
import numpy as np

for tag in tags:
  with open(f'wrime_{tag}_bert_vec.npy', 'wb') as f:
    np.save(f, X[tag])

### 文書分類

In [25]:
import numpy as np

tags = ["train", "validation", "test"]

X = {}
labels = {}
for tag in tags:
  with open(f'wrime_{tag}_bert_vec.npy', 'rb') as f:
    X[tag] = np.load(f)
  with open(f'wrime_{tag}_label.npy', 'rb') as f:
    labels[tag] = np.load(f)

* ラベルを2値に単純化する。先ほどと同じ。つまり・・・
  * ラベル0のテキストは取り除く。
  * negativeを示す-2と-1は、一つのクラスにまとめる。
  * positiveを示す1と2も、一つのクラスにまとめる。

In [27]:
X_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  X_binary[tag] = X[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

* 適宜、チューニングしてください。
* 分類手法は`LinearSVC`でなくても構いません。

In [28]:
from sklearn.svm import LinearSVC

cls = LinearSVC()
cls.fit(X_binary["train"], labels_binary["train"])
cls.score(X_binary["validation"], labels_binary["validation"])

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.8694029850746269

# 本日の課題
* 上で実行した感情分析の性能を上げてください。
* チューニングが済んだら、テストセットでscoreを計算してください。
  * 別の評価尺度で評価してもらっても構いません。